In [1]:
import emoji


In [2]:
emoji.EMOJI_UNICODE

{u':raising_hands_dark_skin_tone:': u'\U0001f64c\U0001f3ff',
 u':Azerbaijan:': u'\U0001f1e6\U0001f1ff',
 u':man_gesturing_NO_medium-light_skin_tone:': u'\U0001f645\U0001f3fc\u200d\u2642\ufe0f',
 u':clapping_hands_medium_skin_tone:': u'\U0001f44f\U0001f3fd',
 u':woman_facepalming_light_skin_tone:': u'\U0001f926\U0001f3fb\u200d\u2640\ufe0f',
 u':man_in_lotus_position_light_skin_tone:': u'\U0001f9d8\U0001f3fb\u200d\u2642\ufe0f',
 u':person_gesturing_NO_medium-light_skin_tone:': u'\U0001f645\U0001f3fc',
 u':baby_medium-light_skin_tone:': u'\U0001f476\U0001f3fc',
 u':microscope:': u'\U0001f52c',
 u':person_walking_medium_skin_tone:': u'\U0001f6b6\U0001f3fd',
 u':women\u2019s_room:': u'\U0001f6ba',
 u':Cuba:': u'\U0001f1e8\U0001f1fa',
 u':pregnant_woman:': u'\U0001f930',
 u':person_mountain_biking_medium_skin_tone:': u'\U0001f6b5\U0001f3fd',
 u':left-facing_fist_medium_skin_tone:': u'\U0001f91b\U0001f3fd',
 u':backhand_index_pointing_down_medium-light_skin_tone:': u'\U0001f447\U0001f3fc',
 u

In [3]:
emoji.emojize(':baseball:')

u'\u26be'

In [4]:
emoji_dictionary={"0":":beating_heart:",
                  "1":":baseball:",
                  "2":":grinning_face_with_big_eyes:",
                  "3":":disappointed_face:",
                  "4":":fork_and_knife:"
                 }

In [5]:
print(emoji_dictionary.values())
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

[':baseball:', ':beating_heart:', ':disappointed_face:', ':grinning_face_with_big_eyes:', ':fork_and_knife:']
⚾
💓
😞
😃
🍴


In [6]:
print(emoji.emojize(":fork_and_knife:"))

🍴


In [7]:
import numpy as np
import pandas as pd

In [8]:
train=pd.read_csv('train_emoji.csv',header=None)
test=pd.read_csv('test_emoji.csv',header=None)
print(train.shape)
print(test.shape)

(132, 4)
(56, 2)


In [9]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [10]:
X_train=train[0]
Y_train=train[1]
X_test=test[0]
Y_test=test[1]
print(X_train.shape)
print(Y_train.shape)

(132,)
(132,)


In [11]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

('never talk to me again', u'\U0001f61e')
('I am proud of your achievements', u'\U0001f603')
('It is the worst day in my life', u'\U0001f61e')
('Miss you so much', u'\U0001f493')
('food is life', u'\U0001f374')


In [12]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
def convert_into_list(sentences):
    total=sentences.shape[0]
    X=[]
    for ix in range(total):
        X.append(sentences[ix].lower().split())
        
    return X    

In [14]:
XT=convert_into_list(X_train)
YT=to_categorical(Y_train)
x_test=convert_into_list(X_test)
y_test=to_categorical(Y_test)

In [15]:
def glove_vec_path(path):
    import numpy as np
    with open(path,'r') as f:
        words = set()
        word_vec = {}
        for line in f:
            line = line.strip().split()
            key = line[0]
            words.add(key)
            val = np.array(line[1:],dtype=np.float64)
            word_vec[key] = val
            
        # Dict (Vocab) --> set
        word2idx = {}
        idx2word = {}
        
        for i,w in enumerate(sorted(words)):
            word2idx[w] = i
            idx2word[i] = w
            
    return word_vec,word2idx,idx2word

In [16]:
 word_vec,word2idx,idx2word=glove_vec_path('glove.6B.50d.txt')

In [17]:
def get_embedding(X):
    embedding_output = np.zeros((len(X),10,50))
    for ix in range(len(X)):
        for jx in range(len(X[ix])):
            embedding_output[ix][jx]=word_vec[X[ix][jx]]
        
    return embedding_output
    

In [18]:
XT_processed=get_embedding(XT)
x_test_processed=get_embedding(x_test)

In [19]:
word_vec["proud"].shape

(50,)

In [20]:
from keras.models import Sequential
from keras.layers import *

In [21]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [22]:
hist=model.fit(XT_processed,YT,batch_size=64,shuffle=True,validation_split=0.2,epochs=100)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 2s 15ms/step - loss: 1.6114 - acc: 0.2571 - val_loss: 1.6195 - val_acc: 0.2963
Epoch 2/100
105/105 [==============================] - 0s 349us/step - loss: 1.5825 - acc: 0.2952 - val_loss: 1.6156 - val_acc: 0.2963
Epoch 3/100
105/105 [==============================] - 0s 409us/step - loss: 1.5503 - acc: 0.3429 - val_loss: 1.6166 - val_acc: 0.2593
Epoch 4/100
105/105 [==============================] - 0s 392us/step - loss: 1.5416 - acc: 0.3429 - val_loss: 1.6228 - val_acc: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 428us/step - loss: 1.5029 - acc: 0.4476 - val_loss: 1.6338 - val_acc: 0.1852
Epoch 6/100
105/105 [==============================] - 0s 434us/step - loss: 1.5146 - acc: 0.3333 - val_loss: 1.6441 - val_acc: 0.2593
Epoch 7/100
105/105 [==============================] - 0s 512us/step - loss: 1.4733 - acc: 0.4190 - val_loss: 1.6524 - val_acc: 0.2593
Epoch 8/100

Epoch 61/100
105/105 [==============================] - 0s 407us/step - loss: 0.0745 - acc: 0.9810 - val_loss: 1.2241 - val_acc: 0.7407
Epoch 62/100
105/105 [==============================] - 0s 422us/step - loss: 0.0561 - acc: 1.0000 - val_loss: 1.4355 - val_acc: 0.7037
Epoch 63/100
105/105 [==============================] - 0s 410us/step - loss: 0.0893 - acc: 0.9810 - val_loss: 1.3548 - val_acc: 0.7407
Epoch 64/100
105/105 [==============================] - 0s 402us/step - loss: 0.0818 - acc: 0.9810 - val_loss: 1.2385 - val_acc: 0.7407
Epoch 65/100
105/105 [==============================] - 0s 388us/step - loss: 0.0487 - acc: 1.0000 - val_loss: 1.2476 - val_acc: 0.6667
Epoch 66/100
105/105 [==============================] - 0s 353us/step - loss: 0.1087 - acc: 0.9810 - val_loss: 1.1511 - val_acc: 0.7037
Epoch 67/100
105/105 [==============================] - 0s 416us/step - loss: 0.0409 - acc: 0.9905 - val_loss: 1.1747 - val_acc: 0.7407
Epoch 68/100
105/105 [==========================

In [29]:
pred=model.predict_classes(x_test_processed)
print(pred)

[4 3 3 0 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 2 0 0 4 0 3 3 2 0 1 2 0 1 0 2 2 1 2
 3 4 2 1 0 0 2 2 0 2 2 3 3 3 3 3 2 3 3]


In [26]:
model.evaluate(x_test_processed,y_test)

56/56 [==============================] - 0s 166us/step


[2.6599908556256975, 0.5892857142857143]

In [32]:
for i in range(30):
    print(' '.join(x_test[i]))
    print(emoji.emojize(emoji_dictionary[str(Y_test[i])]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

i want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😞
she got me a present
💓
💓
ha ha ha it was so funny
😃
😃
he is a good friend
💓
😃
i am upset
💓
😞
we had such a lovely dinner tonight
💓
😃
where is the food
🍴
🍴
stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
this girl is messing with me
😞
💓
are you serious ha ha
😃
😞
let us go play baseball
⚾
⚾
this stupid grader is not working
😞
😞
work is horrible
😞
😃
congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
😃
i love taking breaks
💓
💓
you brighten my day
😃
💓
i boiled rice
🍴
🍴
she is a bully
😞
💓
why are you feeling bad
😞
😞
i am upset
😞
😞
i worked during my birthday
😞
😃
my grandmother is the love of my life
💓
💓
enjoy your break
😃
⚾
valentine day is near
💓
😃
